# Analyze city property inventory

In [1]:
from sqlalchemy import create_engine
import geopandas as gpd
import pandas as pd
from shapely import wkt

In [2]:
DB = create_engine('postgresql://jon@localhost:5432/propertydb')
PUBDATE = pd.to_datetime('2023-08-13')

In [3]:
def ownership_stats(df):
    df['ownershiplength'] = (PUBDATE - pd.to_datetime(df.saledate)).dt.days
    df['ownershiplength'] = (df.ownershiplength / 365).round(1)
    
    ownershiplength = df.ownershiplength.describe().reset_index()
    ownershiplength = ownershiplength.rename(columns={ 'ownershiplength': 'yearcount' })
    ownershiplength['realyear'] = PUBDATE.year - ownershiplength.yearcount

    return ownershiplength

def quick_stats(df):
    df = df.groupby(df.type).aggregate({ 'count': 'sum', 'lotarea': 'sum' }).sort_values(by='count', ascending=False)

    # Convert lot area sqft to acres
    df['lotarea'] = df.lotarea / 43560

    # Percent of agency land
    df['pct'] = df['count'] / df['count'].sum()

    # Percent of city neighborhood acreage
    df['pct_city'] = df.lotarea / 33161.843

    # Multiple of Schenley Park (somewhat recognizable comparison)
    df['mult_schenley'] = df.lotarea / 456

    return df

### City properties by `parceltype`

In [4]:
propertiescitypt = pd.read_sql_query("""SELECT parceltype AS type, COUNT(parceleproppgh.parcelid), SUM(lotarea::int) AS lotarea
FROM parceleproppgh
JOIN assessments
ON assessments.parcelid = parceleproppgh.parcelid
GROUP BY type;""", DB)

# Merge permanent city ownership parcel types
propertiescitypt.loc[propertiescitypt.type.isin([
    'Greenway', 'Park', 'Infrastructure Protection', 'City Facility', 'Other Public Use'
]), 'type'] = 'PCO'

quick_stats(propertiescitypt)

,count,lotarea,pct,pct_city,mult_schenley
type,,,,,
Hold For Study,4991,2748.636042,0.379746,0.082886,6.027711
URA Transfer,2815,200.797153,0.214182,0.006055,0.440345
Public Sale,2712,225.179522,0.206346,0.006790,0.493815
PCO,2240,2287.426102,0.170433,0.068978,5.016285
CDC Property Reserve,192,11.389555,0.014609,0.000343,0.024977
Unknown,180,83.621671,0.013696,0.002522,0.183381
Unknown Public Use,10,0.529982,0.000761,0.000016,0.001162
PLB Transfer,3,0.225367,0.000228,0.000007,0.000494


### URA properties by `parcelstatus`

In [5]:
propertiesuraps = pd.read_sql_query("""SELECT parcelstatus AS type, COUNT(parcelepropura.parcelid), SUM(lotarea::int) AS lotarea
FROM parcelepropura
JOIN assessments
ON assessments.parcelid = parcelepropura.parcelid
GROUP BY parcelstatus;""", DB)

quick_stats(propertiesuraps)

,count,lotarea,pct,pct_city,mult_schenley
type,,,,,
Hold For Study,610,153.682759,0.381966,0.004634,0.337024
To Be Dedicated,204,34.492149,0.127740,0.001040,0.075641
Available,191,55.592860,0.119599,0.001676,0.121914
Disposition in Process,152,732.500230,0.095178,0.022089,1.606360
Site Assemblage,136,139.965266,0.085160,0.004221,0.306941
Exclusive Negotiations,108,16.146350,0.067627,0.000487,0.035409
Acquired,61,8.491414,0.038197,0.000256,0.018622
Hold from Market,57,3.211341,0.035692,0.000097,0.007042
RFP/RFQ Issued,33,3.607163,0.020664,0.000109,0.007910


### City, URA `Hold for Study` and `URA Transfer` properties

In [6]:
stats = quick_stats(pd.concat([
    propertiescitypt[propertiescitypt.type == 'Hold For Study'],
    propertiesuraps[propertiesuraps.type == 'Hold For Study'],
    propertiescitypt[propertiescitypt.type == 'URA Transfer'],
]))
stats.loc['Total'] = stats[stats.columns].sum()

stats

,count,lotarea,pct,pct_city,mult_schenley
type,,,,,
Hold For Study,5601.0,2902.318802,0.665518,0.087520,6.364734
URA Transfer,2815.0,200.797153,0.334482,0.006055,0.440345
Total,8416.0,3103.115955,1.000000,0.093575,6.805079


### Joint city-URA sale date analysis for `Hold For Study`

In [7]:
ownership_stats(pd.read_sql_query("""SELECT saledate
FROM parceleproppgh
JOIN assessments
ON assessments.parcelid = parceleproppgh.parcelid
WHERE parceltype = 'Hold For Study'
AND saledate != ''
UNION ALL
SELECT saledate
FROM parcelepropura
JOIN assessments
ON assessments.parcelid = parcelepropura.parcelid
WHERE parcelstatus = 'Hold For Study'
AND saledate != '';""", DB))

,index,yearcount,realyear
0,count,5579.000000,-3556.000000
1,mean,52.496290,1970.503710
2,std,27.583816,1995.416184
3,min,0.200000,2022.800000
4,25%,32.700000,1990.300000
5,50%,55.300000,1967.700000
6,75%,73.300000,1949.700000
7,max,150.500000,1872.500000


### Sale date analysis for `URA Transfer`

In [8]:
ownership_stats(pd.read_sql_query("""SELECT saledate
FROM parceleproppgh
JOIN assessments
ON assessments.parcelid = parceleproppgh.parcelid
WHERE parceltype = 'URA Transfer'
AND saledate != '';""", DB))

,index,yearcount,realyear
0,count,2815.000000,-792.000000
1,mean,14.660853,2008.339147
2,std,14.971885,2008.028115
3,min,0.200000,2022.800000
4,25%,6.800000,2016.200000
5,50%,10.700000,2012.300000
6,75%,13.300000,2009.700000
7,max,123.100000,1899.900000


### Status of city `URA Transfer` properties

In [9]:
quick_stats(pd.read_sql_query("""SELECT parcelstatus AS type, COUNT(parceleproppgh.parcelid), SUM(lotarea::int) AS lotarea
FROM parceleproppgh
JOIN assessments
ON assessments.parcelid = parceleproppgh.parcelid
WHERE parceltype = 'URA Transfer'
GROUP BY type
ORDER BY count DESC;""", DB))

,count,lotarea,pct,pct_city,mult_schenley
type,,,,,
Available for Sale,2532,176.463131,0.899467,0.005321,0.386981
Hold For Study,142,14.308884,0.050444,0.000431,0.031379
Sale Pending,109,6.697681,0.038721,0.000202,0.014688
Acquisition Pending,29,3.092539,0.010302,0.000093,0.006782
Cancelled,1,0.062098,0.000355,0.000002,0.000136
Redeemed,1,0.114784,0.000355,0.000003,0.000252
Unknown,1,0.058035,0.000355,0.000002,0.000127


### City facilities and `Hold For Study`

In [10]:
facilities = pd.read_sql_query("""SELECT parcelfacilitiespgh.name, parceltype, parcelstatus
FROM parcelfacilitiespgh
JOIN parcelcentroids
ON parcelcentroids.parcelmbl = parcelfacilitiespgh.parcelmbl
LEFT JOIN parceleproppgh
ON parceleproppgh.parcelid = parcelcentroids.parcelid;""", DB)

##### Count of city facilities not in ePropertyPlus

In [11]:
len(facilities[facilities.parceltype.isna()])

21

##### Short list of `Hold For Study` facilities

In [12]:
facilities[facilities.parceltype == 'Hold For Study'].sort_values(by='name').head(10)

,name,parceltype,parcelstatus
98,Albert Turk Graham Park Shelter,Hold For Study,Hold For Study
81,Allegheny Northside Senior Center and Hazlett ...,Hold For Study,Hold For Study
282,Ammon Recreation Center,Hold For Study,Hold For Study
354,Asphalt Plant Equipment Shelter,Hold For Study,Hold For Study
356,Asphalt Plant Garage 2,Hold For Study,Hold For Study
355,Asphalt Plant Testing Lab Building,Hold For Study,Hold For Study
101,Banksville Concession Stand,Hold For Study,Hold For Study
100,Banksville School Field 3rd Base Dugout,Hold For Study,Hold For Study
93,Banksville School Field Storage Building,Hold For Study,Hold For Study
75,Beechview Senior and Community Center,Hold For Study,Hold For Study


##### Facilities by parcel type

In [13]:
facilitiesparceltype = facilities.parceltype.value_counts().reset_index()
facilitiesparceltype['pct'] = facilitiesparceltype['count'] / facilitiesparceltype['count'].sum()
facilitiesparceltype

,parceltype,count,pct
0,Park,191,0.498695
1,Hold For Study,184,0.480418
2,City Facility,4,0.010444
3,URA Transfer,2,0.005222
4,Greenway,1,0.002611
5,Public Sale,1,0.002611


### City green space and `Hold For Study`

In [14]:
green = gpd.GeoDataFrame(
    pd.concat([
        gpd.read_file('input/greenways.geojson'),
        gpd.read_file('input/parks.geojson')
    ])
)

green['name'] = green.name.fillna(green.origpkname)

In [15]:
points = gpd.read_postgis("""SELECT parcelcentroids.parcelid, parceltype, ST_MakePoint(lon::float, lat::float) AS geom
FROM parceleproppgh
JOIN parcelcentroids
ON parcelcentroids.parcelid = parceleproppgh.parcelid;""", DB, crs='EPSG:4269')
points = points.to_crs('EPSG:4326')

##### Join properties to green space

In [16]:
pointsgreen = gpd.sjoin(points, green, predicate='within', how='inner')

In [17]:
pointsgreenparceltype = pointsgreen.parceltype.value_counts().reset_index()
pointsgreenparceltype['pct'] = pointsgreenparceltype['count'] / pointsgreenparceltype['count'].sum()
pointsgreenparceltype

,parceltype,count,pct
0,Greenway,1039,0.421501
1,Park,859,0.348479
2,Hold For Study,536,0.217444
3,URA Transfer,15,0.006085
4,Public Sale,10,0.004057
5,Infrastructure Protection,4,0.001623
6,City Facility,1,0.000406
7,Unknown,1,0.000406


##### `Hold For Study` properties by green name

In [18]:
pointsgreenparkname = pointsgreen[pointsgreen.parceltype == 'Hold For Study'].name.value_counts().reset_index()
pointsgreenparkname['pct'] = pointsgreenparkname['count'] / pointsgreenparkname['count'].sum()
pointsgreenparkname

,name,count,pct
0,Brookline Memorial Park,81,0.155769
1,Bigelow Park,53,0.101923
2,Hazelwood Park,44,0.084615
3,Duquesne Heights Greenway,39,0.075000
4,Bigelow,28,0.053846
...,...,...,...
71,Allegheny Landing Park,1,0.001923
72,Mellon Park,1,0.001923
73,Dunseith Tot Lot (Shalane's Play Yard),1,0.001923
74,Marmaduke Playground,1,0.001923


### Joint facilities-green space and `Hold For Study`

In [19]:
pd.concat([
    facilitiesparceltype[facilitiesparceltype.parceltype == 'Hold For Study'],
    pointsgreenparceltype[pointsgreenparceltype.parceltype == 'Hold For Study']
])['count'].sum()

720

### Joint city-URA zoning for `Hold For Study`

In [20]:
zoning = gpd.read_file('input/zoning.geojson')

In [21]:
h4spoints = gpd.read_postgis("""SELECT parcelcentroids.parcelid, parcelboundaries.wkt AS wkt_boundaries, ST_MakePoint(lon::float, lat::float) AS geom
FROM parceleproppgh
JOIN parcelcentroids
ON parcelcentroids.parcelid = parceleproppgh.parcelid
JOIN parcelboundaries
ON parcelboundaries.parcelid = parceleproppgh.parcelid
WHERE parceltype = 'Hold For Study'
UNION ALL
SELECT parcelcentroids.parcelid, parcelboundaries.wkt AS wkt_boundaries, ST_MakePoint(lon::float, lat::float) AS geom
FROM parcelepropura
JOIN parcelcentroids
ON parcelcentroids.parcelid = parcelepropura.parcelid
JOIN parcelboundaries
ON parcelboundaries.parcelid = parcelepropura.parcelid
WHERE parcelstatus = 'Hold For Study';""", DB, crs='EPSG:4269')
h4spoints['geom_boundaries'] = h4spoints.wkt_boundaries.apply(wkt.loads)
h4spoints = h4spoints.to_crs('EPSG:4326')

In [22]:
h4spointszoning = gpd.sjoin(h4spoints, zoning, predicate='within', how='inner')

In [23]:
h4spointszoningtype = h4spointszoning.legendtype.value_counts().reset_index()
h4spointszoningtype['pct'] = h4spointszoningtype['count'] / h4spointszoningtype['count'].sum()
h4spointszoningtype

,legendtype,count,pct
0,Hillside,2055,0.366964
1,Parks,1140,0.203571
2,Single-Unit Detached Residential,679,0.121250
3,Multi-Unit Residential,543,0.096964
4,Two-Unit Residential,492,0.087857
5,Single-Unit Attached Residential,263,0.046964
6,Local Neighborhood Commercial,136,0.024286
7,Urban Industrial,78,0.013929
8,Riverfront,55,0.009821
9,Uptown Public Realm,41,0.007321


### City Council and `Hold For Study`

In [24]:
council = gpd.read_file('input/council.geojson')

In [25]:
h4spointscouncil = gpd.sjoin(h4spoints, council, predicate='within', how='inner')

In [26]:
h4spointscounciltype = h4spointscouncil.DIST_NAME.value_counts().reset_index()
h4spointscounciltype['pct'] = h4spointscounciltype['count'] / h4spointscounciltype['count'].sum()
h4spointscounciltype

,DIST_NAME,count,pct
0,D6,1184,0.211353
1,D2,1032,0.184220
2,D9,830,0.148161
3,D1,819,0.146198
4,D5,628,0.112103
5,D3,481,0.085862
6,D4,424,0.075687
7,D7,188,0.033559
8,D8,16,0.002856


### City Council and `URA Transfer`

In [27]:
uraxferpoints = gpd.read_postgis("""SELECT parcelcentroids.parcelid, parcelboundaries.wkt AS wkt_boundaries, ST_MakePoint(lon::float, lat::float) AS geom
FROM parceleproppgh
JOIN parcelcentroids
ON parcelcentroids.parcelid = parceleproppgh.parcelid
JOIN parcelboundaries
ON parcelboundaries.parcelid = parceleproppgh.parcelid
WHERE parceltype = 'URA Transfer';""", DB, crs='EPSG:4269')
uraxferpoints['geom_boundaries'] = uraxferpoints.wkt_boundaries.apply(wkt.loads)
uraxferpoints = uraxferpoints.to_crs('EPSG:4326')

In [28]:
uraxferpointscouncil = gpd.sjoin(uraxferpoints, council, predicate='within', how='inner')

In [29]:
uraxferpointscounciltype = uraxferpointscouncil.DIST_NAME.value_counts().reset_index()
uraxferpointscounciltype['pct'] = uraxferpointscounciltype['count'] / uraxferpointscounciltype['count'].sum()
uraxferpointscounciltype

,DIST_NAME,count,pct
0,D9,1334,0.473553
1,D6,737,0.261626
2,D3,294,0.104366
3,D5,283,0.100461
4,D2,91,0.032304
5,D1,44,0.015619
6,D7,29,0.010295
7,D4,5,0.001775
